In [33]:
from __future__ import division
import numpy as np
import matplotlib.pyplot as plt
import pylab

In [34]:
M = 1     
N = 100

def calc_g_r(name, L):          
    NP = 64
    r = np.linspace(0.001,L/2,50)
    
    dist = []
    
    for run in range(10):
        dis = np.zeros(len(r))
        for i in range(M,N):
            data=pylab.loadtxt('./rod/'+str(run)+'_'+name+'_'+str(round(L,2))+'/traj_MC_'+name+'_'+str(round(L,2))+'_'+str(i)+'.dat')
            x=data[:,0]
            y=data[:,1]
            for j in range(0,NP):
                for k in range(j,NP):
                    dx = abs(x[j]-x[k])
                    dy = abs(y[j]-y[k])

                    #periodic boundary condition
                    if dx>L/2: dx = L-dx
                    if dy>L/2: dy = L-dy

                    d=((dx)**2+(dy)**2)**0.5

                    #count the number of particles from r to r+1
                    for l in range(0,len(r)-1):
                        if d>r[l] and d<r[l+1]:
                            dis[l+1]=dis[l+1]+1
                            
        for i in range(0,len(dis)):
            dis[i]=dis[i]/((N-M)*2*3.14*r[i])
            dis[i]=dis[i]/((64/2)*(r[2]-r[1])*(64/(L*L)))
            
        dist.append(dis)
        
    std = np.std(dist, axis=0)
    dist = np.average(dist,axis=0)
    
    return r,dist,std

In [35]:
Area_frac = 0.2
D = 0.11111111111111
num_disk = 32
phi = 4*Area_frac/(num_disk*np.pi*D*D)

NP = 64
L = (NP/phi)**0.5

r,dis,std=calc_g_r('NORMAL', L)
# plt.plot(r,dis,color='red', label='Explicit Distance Calculation')
# plt.scatter(r,dis)

r2,dis2,std2=calc_g_r('ML_Grad_Boost', L)
# plt.plot(r,dis, color='black',label='Grad Boost')
# plt.scatter(r,dis)

In [ ]:
plt.errorbar(r,dis,yerr=std,ecolor='red',elinewidth=1,capsize=5,color='red', label='Explicit Distance Calculation')
plt.errorbar(r2,dis2,yerr=std2,ecolor='blue',elinewidth=1,capsize=5,color='blue',label='ML Model')

plt.title('Rod', fontsize=15)
plt.legend(loc=4, fontsize=15)
plt.xlabel(r"$r$",fontsize=15)
plt.ylabel(r"$g(r)$",fontsize=15)
plt.tick_params(direction='in', length=6, width=2, colors='black', labelsize=15, grid_color='black')

plt.tight_layout()
plt.savefig('../snaps/rod_0.2_with_error_bar_up.png', dpi=200)

plt.show()